In [ ]:
import os

PATH = os.path.abspath("test.ipynb")
DATA = os.path.join(os.path.dirname(PATH), "data/output_optical_7.txt")

f = open(DATA)

In [ ]:
# Store metadata as strings (for now).
metadata = {}

for i in range(2):
    devnull = f.readline() # Ignore the first two lines.

while True:
    x = f.tell() # Define a stopping point for later (a bit dirty but it's not too bad).
    l = f.readline()
    if l[3] != '>': break # From this point we begin reading events.
    sl = l.split(' ')
    if len(sl) == 7:
        metadata[sl[5]] = sl[6][:-1]
devnull = f.seek(x)

In [ ]:
# Store event data in "struct":
#     event_data[0] : header bank (10).
#     event_data[1] : user header bank (currently empty). Assumed to have same format as header bank.
#     event_data[2] : integrated raw bank (51).
#     event_data[3] : integrated digitized bank (52).
#     event_data[4] : generated particles bank.
event_data = ({}, {}, {}, {}, {})
bcounter = -1 # Bank counter.
eof = 0 # End of file checker.

while True:
    l = f.readline()
    l = l[:-1]
    if l == " ---- End of Event  ----": 
        break
    if l == '':
        eof = 1
        break

    # Make sure we're writing to the correct address.
    if   l == " --- Header Bank --":                          bcounter = 0
    elif l == " --- User Header Bank --":                     bcounter = 1
    elif l == "   -- integrated true infos bank  (51, 0) --": bcounter = 2
    elif l == "   -- integrated digitized bank  (52, 0) --":  bcounter = 3
    elif l == " --- Generated Particles Bank --":             bcounter = 4

    if bcounter != 4:
        sl = l.split('\t')
        if len(sl) == 1: continue # Ignore lines with titles & additional information.

    if bcounter < 2: # Header & user header banks.
        event_data[bcounter][sl[0].split(' ')[-1][:-1]] = sl[1]
    elif bcounter < 4: # Raw & Digitized banks.
        event_data[bcounter][sl[0].split(' ')[-1][:-1]] = sl[1:]
    if bcounter == 4: # Generated particles bank.
        sl = l.split()
        if sl[1] == 'Particle':
            event_data[bcounter][sl[ 3][:-1]] = sl[ 4]
            event_data[bcounter][sl[ 6][:-1]] = sl[ 7]
            event_data[bcounter][sl[10][:-1]] = sl[11]
        if sl[1] == 'Hit':
            event_data[bcounter]['nhits'] = sl[ 4]
            event_data[bcounter][sl[ 7]]  = sl[ 8]
            event_data[bcounter][sl[11]]  = sl[12]

if eof == 0: print("Event #%s stored!" % (event_data[0]['evn']))
else: print("Reached end of file!")

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

pp.pprint(event_data)